### Change Directory

In [ ]:
cd "C:/Users/Karpoh/Desktop/Code/DeepLearnAss2/Inference"

In [ ]:
pwd

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')


In [ ]:
#cd /content/drive/MyDrive/UCCD3074_GroupAssignment/Final/

### Import Library

In [ ]:
import numpy as np
import cv2
import torch
import glob as glob
import pandas as pd
import os
import albumentations as A
import time
from albumentations.pytorch import ToTensorV2
from torch.nn import functional as F
from torch import topk
from torch.nn.modules.module import register_module_forward_hook
import torchvision.models as models
from torchsummary import summary
import matplotlib.pyplot as plt
from torchvision.models import resnet18, ResNet18_Weights, ResNet50_Weights

imgSize = 224

#from model import build_model
# Define computation device.
device = 'cpu'
# Class names.
sign_names_df = pd.read_csv('signnames.csv')
class_names = sign_names_df.SignName.tolist()

# # DataFrame for ground truth.
# gt_df = pd.read_csv(
#     'Inference/Test.csv', 
#     delimiter=';'
# )
# gt_df = gt_df.set_index('Filename', drop=True)

#Read different testing folder
gt_df = pd.read_csv('../Test.csv')
gt_df = gt_df.set_index('Path', drop=True)

# Initialize model, switch to eval model, load trained weights.
model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
#model = models.inception_v3(pretrained=True)
#model = models.alexnet(pretrained=True)

#Modify the fully connected layer
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, 43)

#num_features = model.classifier[6].in_features
#model.fc = torch.nn.Linear(num_features, 43)
model.load_state_dict(torch.load("../models/PreTrained_ResNet18_v3.6.pth"))
#model.load_state_dict(torch.load("models/PreTrained_Inception_v1.0.pth"))
#model.load_state_dict(torch.load("models/PreTrained_AlexNet_v1.0.pth"))
model = model.eval()

In [ ]:
# for name, _ in model.named_children():
#   print(name)

# print(model)

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# summary(model, (3,224,224))

### Define Function To Print HeatMap

In [ ]:
def returnCAM(feature_conv, weight_softmax, class_idx):
    # Generate the class activation maps upsample to 256x256.
    size_upsample = (256, 256)
    bz, nc, h, w = feature_conv.shape
    output_cam = []
    for idx in class_idx:
        #print(weight_softmax[idx].shape)
        cam = weight_softmax[idx].dot(feature_conv.reshape((nc, h*w)))

        #print('cam', cam)
        #print('shape', feature_conv.shape)
        cam = cam.reshape(h, w)
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(cv2.resize(cam_img, size_upsample))
    return output_cam

In [ ]:
def apply_color_map(CAMs, width, height, orig_image):
    for i, cam in enumerate(CAMs):
        heatmap = cv2.applyColorMap(cv2.resize(cam,(width, height)), cv2.COLORMAP_JET)
        result = heatmap * 0.5 + orig_image * 0.5
        result = cv2.resize(result, (imgSize, imgSize))
        return result

In [ ]:
def visualize_and_save_map(result, orig_image, gt_idx=None, class_idx=None, save_name=None):
    # Put class label text on the result.
    if class_idx is not None:
        cv2.putText(
            result, 
            f"Pred: {str(class_names[int(class_idx)])}", (5, 20), 
            cv2.FONT_HERSHEY_SIMPLEX, 0.55, (0, 255, 0), 2,
            cv2.LINE_AA
        )
    if gt_idx is not None:
        cv2.putText(
            result, 
            f"GT: {str(class_names[int(gt_idx)])}", (5, 40), 
            cv2.FONT_HERSHEY_SIMPLEX, 0.55, (0, 255, 0), 2,
            cv2.LINE_AA
        )
    
    ####################
    # Resize the original image
    orig_image = cv2.resize(orig_image, (imgSize, imgSize))

    # Combine the result and original images side-by-side
    img_concat = np.concatenate([result, orig_image], axis=1)

    # Display the result using pyplot
    plt.imshow(img_concat/255.)
    plt.title('Result')
    plt.axis('off')
    plt.show()
    ####################

In [ ]:
#summary(model, (3,244,244))

In [ ]:
# Hook the feature extractor.
# https://github.com/zhoubolei/CAM/blob/master/pytorch_CAM.py
features_blobs = []
def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())

#Resnet
model.layer4.register_forward_hook(hook_feature) 

#Inception
#model.AuxLogits.register_forward_hook(hook_feature) 

#AlexNet
#model.features.register_forward_hook(hook_feature) 

# Get the softmax weight.
params = list(model.parameters())
weight_softmax = np.squeeze(params[-2].data.cpu().numpy())
#print(weight_softmax.shape)
#print(weight_softmax)
# Define the transforms, resize => tensor => normalize.
transform = A.Compose([
    A.Resize(imgSize, imgSize),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    ToTensorV2(),
    ])

## Inference and Print HeatMap

### ResNet

In [ ]:
################################################################################################
# Reference: https://github.com/zhoubolei/CAM/blob/master/pytorch_CAM.py
#################################################################################################

In [ ]:
incorrect = []

In [ ]:
counter = 0
true_labels = []
pred_labels = []
# Run for all the test images.
#all_images = glob.glob('/content/drive/MyDrive/UCCD3074/TrafficSignRecognition/GTSRB/Final_Test/Images/*.ppm')
## Images from different test folder
all_images = glob.glob('../Test/*.png')

correct_count = 0
frame_count = 0 # To count total frames.
total_fps = 0 # To get the final frames per second. 
for i, image_path in enumerate(all_images):
    # Read the image.
    image = cv2.imread(image_path)
    orig_image = image.copy()
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height, width, _ = orig_image.shape
    # Apply the image transforms.
    image_tensor = transform(image=image)['image']
    # Add batch dimension.
    image_tensor = image_tensor.unsqueeze(0)
    # Forward pass through model.
    start_time = time.time()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # image_tensor = image_tensor.to(device)
    # outputs = model(image_tensor) 
    
    model = model.to(device)
    outputs = model(image_tensor.to(device))
    end_time = time.time()
    # Get the softmax probabilities.
    probs = F.softmax(outputs, dim=1).data.squeeze()
    # Get the class indices of top k probabilities.
    class_idx = topk(probs, 1)[1].int()
    #print(class_idx)
    # Get the ground truth.
    image_name = image_path.split(os.path.sep)[-1]
    ## Different Testing Folder
    image_name = "Test/" + image_name
    gt_idx = gt_df.loc[image_name].ClassId
    # Check whether correct prediction or not.
    if gt_idx == class_idx:
        correct_count += 1

    # Append to true_labels and pred_labels
    true_labels.append(gt_idx)
    pred_labels.append(class_idx)

    # Generate class activation mapping for the top1 prediction.
    CAMs = returnCAM(features_blobs[0], weight_softmax, class_idx)
    # File name to save the resulting CAM image with.
    save_name = f"{image_path.split('/')[-1].split('.')[0]}"
    # Show and save the results.
    result = apply_color_map(CAMs, width, height, orig_image)
#     if(gt_idx == 23 and gt_idx != class_idx):
#         visualize_and_save_map(result, orig_image, gt_idx, class_idx, save_name)
    visualize_and_save_map(result, orig_image, gt_idx, class_idx, save_name)
    if gt_idx != class_idx:
        incorrect.append(gt_idx)
    counter += 1
    #print(f"Image: {counter}")
    # Get the current fps.
    #fps = 1 / (end_time - start_time)
    # Add `fps` to `total_fps`.
    #total_fps += fps
    # Increment frame count.
    #frame_count += 1
print(f"Total number of test images: {len(all_images)}")
print(f"Total correct predictions: {correct_count}")
print(f"Accuracy: {correct_count/len(all_images)*100:.3f}")

# calculate and print the average FPS
#avg_fps = total_fps / frame_count
#print(f"Average FPS: {avg_fps:.3f}")

# Print Accuracy without HeatMap

In [ ]:

# Run for all the test images.
#all_images = glob.glob('/content/drive/MyDrive/UCCD3074/TrafficSignRecognition/GTSRB/Final_Test/Images/*.ppm')
all_images = glob.glob('../Test/*.png')
correct_count = 0
total = 0 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for i, image_path in enumerate(all_images):
    # Read the image.
    image = cv2.imread(image_path)
    orig_image = image.copy()
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height, width, _ = orig_image.shape
    # Apply the image transforms.
    image_tensor = transform(image=image)['image']
    # Add batch dimension.
    image_tensor = image_tensor.unsqueeze(0)
   
    model = model.to(device)
    outputs = model(image_tensor.to(device))

    # Get the softmax probabilities.
    probs = F.softmax(outputs, dim=1).data.squeeze()
    # Get the class indices of top k probabilities.
    class_idx = topk(probs, 1)[1].int()
    # Get the ground truth.
    image_name = image_path.split(os.path.sep)[-1]
    image_name = "Test/" + image_name
    gt_idx = gt_df.loc[image_name].ClassId
    # Check whether correct prediction or not.
    if gt_idx == class_idx:
        correct_count += 1
    
    if gt_idx != class_idx:
        incorrect.append(gt_idx)
    
    total+=1

print("Total correct : %d/%d \nAccuracy: %f" %(correct_count, total, correct_count/total))

In [ ]:
# Create an empty dictionary to store counts of each number
counts = {}

# Loop through each number in the list and update the dictionary
for num in incorrect:
    if num in counts:
        counts[num] += 1
    else:
        counts[num] = 1

# Print the counts of each number in ascending order
for num, count in sorted(counts.items()):
    print(num, ":", count)

In [ ]:
# Print classification report
from sklearn.metrics import classification_report

# Convert pred_labels from list to tensor
pred_labels_tensor = torch.tensor(pred_labels)
true_labels_tensor = torch.tensor(true_labels)

# Assuming true_labels is a tensor
true_labels_cpu = true_labels_tensor.cpu()

# Copy the predicted labels tensor to the CPU
pred_labels_cpu = pred_labels_tensor.cpu()

# Pass CPU tensors to classification_report function
print(classification_report(true_labels_cpu, pred_labels_cpu))

In [ ]:
from sklearn.metrics._plot.confusion_matrix import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import itertools
from sklearn.metrics import confusion_matrix

# Assuming true_labels is a tensor
true_labels_cpu = true_labels_tensor.cpu()

# Copy the predicted labels tensor to the CPU
pred_labels_cpu = pred_labels_tensor.cpu()

# Define classes
classes = np.unique(true_labels_cpu)
classes = [str(c) for c in classes]

# Get the confusion matrix
conf_matrix = confusion_matrix(true_labels_cpu, pred_labels_cpu)

# Calculate percentage of predicted for each class
conf_matrix_percent = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]

cm_display = ConfusionMatrixDisplay(confusion_matrix=conf_matrix_percent, display_labels=classes)

fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(1,1,1)

cm_display.plot(values_format='.1f', cmap='Greens', ax=ax)

plt.title('Confusion Matrix')
plt.show()


In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from itertools import cycle
import matplotlib.pyplot as plt
import random

# Binarize the labels
n_classes = 43
true_labels_np = true_labels_cpu.numpy()
true_labels_onehot = label_binarize(true_labels_np, classes=np.arange(n_classes))

# Binarize the predicted labels
pred_labels_onehot = label_binarize(pred_labels_cpu.numpy(), classes=np.arange(n_classes))


# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
colours = []


for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(true_labels_onehot[:, i], pred_labels_onehot[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    colour = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    # Convert the RGB tuple to a hex string
    colour_hex = '#{:02x}{:02x}{:02x}'.format(*colour)
    colours.append(colour_hex)

    
# Plot ROC curve
plt.figure(figsize=(10, 10))
for i, color in zip(range(n_classes), colours):
    plt.plot(fpr[i], tpr[i], color=color, lw=2, label='ROC curve of class {0} (AUC = {1:0.2f})'
                                               ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.legend(loc="lower right")
plt.show()


In [ ]:
# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(true_labels_onehot.ravel(), pred_labels_onehot.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])


# Compute macro-average ROC curve and ROC area
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at these points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes
fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])


# Plot micro-average ROC curve
plt.figure(figsize=(10, 10))
plt.plot(fpr["micro"], tpr["micro"], label='micro-average ROC curve (AUC = {0:0.2f})'
                                             ''.format(roc_auc["micro"]), color='deeppink', linestyle=':', linewidth=4)
plt.plot(fpr["macro"], tpr["macro"], label='macro-average ROC curve (AUC = {0:0.2f})'
                                             ''.format(roc_auc["macro"]), color='navy', linestyle=':', linewidth=4)
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve for Micro & Macro-average')
plt.legend(loc="lower right")
plt.show()

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

# Compute precision-recall curve and average precision score for each class
precision = dict()
recall = dict()
average_precision = dict()
colours = []

for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(true_labels_onehot[:, i], pred_labels_onehot[:, i])
    average_precision[i] = average_precision_score(true_labels_onehot[:, i], pred_labels_onehot[:, i])
    colour = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    # Convert the RGB tuple to a hex string
    colour_hex = '#{:02x}{:02x}{:02x}'.format(*colour)
    colours.append(colour_hex)

# Plot precision-recall curve
plt.figure(figsize=(10, 10))
for i, color in zip(range(n_classes), colours):
    plt.plot(recall[i], precision[i], color=color, lw=2, label='PR curve of class {0} (avg = {1:0.2f})'
                                               ''.format(i, average_precision[i]))

plt.xlim([0.0, 1.05])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")
plt.show()
